## Unit 09. 綜合案例研究 課堂作業
- 課程編號: CHEXXXX
- 學年度: 114下
- 上課時間: 每週四 09:00-12:00
-
- 指導教授: ＯＯＯ 教授
- 學生姓名: ＯＯＯ
- 學號: m12345678
- email帳號: fcu.m12345678@gmail.com

---
### 環境設定

In [ ]:
from pathlib import Path
import os

# ========================================
# 路徑設定 (兼容 Colab 與 Local)
# ========================================
UNIT_OUTPUT_DIR = 'P2_Unit09_HW'
SOURCE_DATA_DIR = 'TEP_data'

try:
  from google.colab import drive
  IN_COLAB = True
  print("✓ 偵測到 Colab 環境，準備掛載 Google Drive...")
  drive.mount('/content/drive', force_remount=True)
except ImportError:
  IN_COLAB = False
  print("✓ 偵測到 Local 環境")
try:
  shortcut_path = '/content/CHE-AI-COURSE'
  os.remove(shortcut_path)
except FileNotFoundError:
  pass

if IN_COLAB:
  source_path = Path('/content/drive/My Drive/Colab Notebooks/CHE-AI-COURSE')
  os.symlink(source_path, shortcut_path)
  shortcut_path = Path(shortcut_path)
  if source_path.exists():
    NOTEBOOK_DIR = shortcut_path / 'Part_2' / 'Unit09'
    OUTPUT_DIR = NOTEBOOK_DIR / 'outputs' / UNIT_OUTPUT_DIR
    DATA_DIR = NOTEBOOK_DIR / 'data' / SOURCE_DATA_DIR
    MODEL_DIR = OUTPUT_DIR / 'models'
    FIG_DIR = OUTPUT_DIR / 'figs'
  else:
    print(f"⚠️ 找不到路徑雲端CHE-AI-COURSE路徑，請確認自己的雲端資料夾是否正確")
  
else:
  NOTEBOOK_DIR = Path.cwd()
  OUTPUT_DIR = NOTEBOOK_DIR / 'outputs' / UNIT_OUTPUT_DIR
  DATA_DIR = NOTEBOOK_DIR / 'data' / SOURCE_DATA_DIR
  MODEL_DIR = OUTPUT_DIR / 'models'
  FIG_DIR = OUTPUT_DIR / 'figs'

NOTEBOOK_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
DATA_DIR.mkdir(parents=True, exist_ok=True)
MODEL_DIR.mkdir(parents=True, exist_ok=True)
FIG_DIR.mkdir(parents=True, exist_ok=True)

print(f"\n✓ Notebook工作目錄: {NOTEBOOK_DIR}")
print(f"✓ 數據來源目錄: {DATA_DIR}")
print(f"✓ 結果輸出目錄: {OUTPUT_DIR}")
print(f"✓ 模型輸出目錄: {MODEL_DIR}")
print(f"✓ 圖檔輸出目錄: {FIG_DIR}")

---
### 載入相關套件

In [ ]:
# 基礎套件
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# sklearn 套件 - 資料處理
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score

# sklearn 套件 - 分群模型
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.mixture import GaussianMixture

# sklearn 套件 - 降維模型
from sklearn.decomposition import PCA, KernelPCA
from sklearn.manifold import TSNE
import umap

# sklearn 套件 - 異常檢測模型
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.neighbors import LocalOutlierFactor
from sklearn.covariance import EllipticEnvelope

# 關聯規則學習
from mlxtend.frequent_patterns import apriori, fpgrowth, association_rules
from mlxtend.preprocessing import TransactionEncoder

# 設定隨機種子以確保結果可重現
SEED = 42
np.random.seed(SEED)

# 設定matplotlib中文顯示
plt.rcParams['axes.unicode_minus'] = False

print("✓ 所有套件已成功載入")

---
### **I. 練習題: 田納西-伊士曼製程 (TEP) 綜合數據分析**

本作業將使用田納西-伊士曼製程 (Tennessee Eastman Process, TEP) 的真實模擬數據，綜合應用本課程所學的所有非監督式學習方法，進行完整的製程數據分析。

**背景說明**：
- TEP 是化工製程控制領域的標準測試案例
- 包含 52 個變數：41 個測量變數 + 11 個操作變數
- 提供正常操作數據 (IDV=0) 和 21 種故障場景數據 (IDV=1~21)
- 數據已預先處理並存放在 `data/TEP_data/` 資料夾中

**數據檔案**：
- `d00.dat` / `d00_te.dat`: 正常操作訓練/測試數據
- `d01.dat` / `d01_te.dat` ~ `d21.dat` / `d21_te.dat`: 各種故障場景數據

### **1. 數據載入與探索性分析**

**要求**：
- 載入正常操作數據 (d00.dat) 和至少 2 種故障場景數據 (自行選擇 IDV 1~21 中的任意 2 種)
- 檢查數據形狀、缺失值、基本統計資訊
- 繪製至少 5 個關鍵變數的時間序列圖，比較正常與故障狀態
- 繪製相關係數熱力圖，找出高度相關的變數組
- 數據標準化處理 (StandardScaler)

**提示**：
- TEP 數據格式為空格分隔的文本檔 (.dat)
- 可使用 `pd.read_csv(file_path, sep='\s+', header=None)` 讀取
- 變數名稱可參考 TEP 文獻或課堂講義

In [ ]:
# 你的程式碼



### **2. 降維與視覺化分析**

**要求**：
- 使用 **PCA** 降至 2 維和 3 維，繪製散點圖
  - 標註正常操作與不同故障類型的數據點（使用不同顏色）
  - 計算並顯示前 3 個主成分的解釋變異量
  - 繪製主成分貢獻圖 (Loading Plot)，找出對第一、第二主成分貢獻最大的變數
- 使用 **t-SNE** 降至 2 維，繪製散點圖
- 使用 **UMAP** 降至 2 維，繪製散點圖
- 比較三種降維方法的視覺化效果，分析哪種方法最能區分正常與故障狀態

**提示**：
- 先對數據進行標準化
- t-SNE 和 UMAP 計算時間較長，可使用較小的樣本數進行測試
- 注意設定 `random_state=SEED` 確保結果可重現

In [ ]:
# 你的程式碼



### **3. 操作模式識別 (分群分析)**

**要求**：
- 僅使用正常操作數據 (d00.dat)，嘗試識別不同的操作模式
- 使用至少 3 種分群方法：
  - **K-Means**：使用肘部法則和輪廓係數確定最佳 k 值
  - **階層式分群 (Hierarchical Clustering)**：繪製樹狀圖 (Dendrogram)
  - **DBSCAN** 或 **高斯混合模型 (GMM)**：自行選擇一種
- 計算每種方法的評估指標：
  - Silhouette Score
  - Davies-Bouldin Index
  - Calinski-Harabasz Index
- 將分群結果投影到 PCA 2D 空間進行視覺化
- 分析每個群集的特徵統計量，解釋不同操作模式的物理意義

**提示**：
- 正常操作數據可能包含多種穩態操作模式 (如不同的產品比例)
- 可使用 PCA 降維後的數據進行分群，加快計算速度
- 比較不同方法識別出的群集是否一致

In [ ]:
# 你的程式碼



### **4. 異常檢測與故障診斷**

**要求**：
- 使用正常操作數據 (d00.dat) 訓練異常檢測模型
- 建立至少 2 種異常檢測模型：
  - **Isolation Forest**
  - **One-Class SVM** 或 **LOF** 或 **Elliptic Envelope**：自行選擇一種
- 在測試數據上進行異常檢測：
  - 正常操作測試數據 (d00_te.dat)
  - 至少 2 種故障場景測試數據 (d0X_te.dat)
- 計算每種模型在不同數據集上的異常檢測率 (Anomaly Rate)
- 繪製異常分數 (Anomaly Score) 的時間序列圖
- 分析哪種模型最能有效區分正常與故障狀態

**提示**：
- 異常檢測模型只用正常數據訓練
- 故障數據應被識別為異常 (負標籤 = -1)
- 可調整 `contamination` 參數以優化檢測性能
- 比較不同故障類型的檢測難度

In [ ]:
# 你的程式碼



### **5. PCA-based MSPC 統計製程監控 (進階)**

**要求**：
- 使用正常操作數據建立 PCA 模型
- 計算 **T² 統計量** 和 **SPE (Q 統計量)**
- 根據卡方分布和 F 分布計算 95% 和 99% 信賴水準的控制限
- 在測試數據上繪製 T² 和 SPE 監控圖 (Control Chart)
- 分析哪些時間點超出控制限 (異常點)
- 對異常點進行**貢獻圖分析** (Contribution Plot)，找出導致異常的關鍵變數

**提示**：
- T² 統計量衡量數據在主成分空間中的異常程度
- SPE 統計量衡量數據在殘差空間中的異常程度
- 貢獻圖可以幫助識別故障根因
- 參考課堂講義中的 MSPC 公式

In [ ]:
# 你的程式碼



### **6. 綜合分析報告**

**要求**：
撰寫一份簡要的分析報告 (使用 Markdown cell)，內容包括：

1. **數據概述**：描述使用的數據集和基本特徵
2. **降維分析**：總結 PCA、t-SNE、UMAP 的視覺化結果，比較優劣
3. **分群分析**：總結不同分群方法識別出的操作模式，解釋其物理意義
4. **異常檢測**：總結不同異常檢測方法的性能，分析最有效的方法
5. **MSPC 監控**：分析 T² 和 SPE 的監控結果，識別關鍵變數
6. **工程建議**：根據分析結果，提出 2~3 條製程改善或監控策略建議

**提示**：
- 報告應簡潔明瞭，重點突出
- 使用圖表支持結論
- 結合化工領域知識進行解釋

## 綜合分析報告

### 1. 數據概述
(請在此填寫你的分析)

### 2. 降維分析
(請在此填寫你的分析)

### 3. 分群分析
(請在此填寫你的分析)

### 4. 異常檢測
(請在此填寫你的分析)

### 5. MSPC 監控
(請在此填寫你的分析)

### 6. 工程建議
(請在此填寫你的建議)

---
## II. 額外加分作業 (自由繳交)

- 學生可自由選擇是否繳交加分作業 (下週上課前完成 email 電子檔)
- 分數會加在最後總成績上，每份作業加 0.1 ~ 1.0 分
- 分數加的不多，真的不一定要交加分作業，正常出席上課做好期末報告即可
- 加分作業不一定要全部都做完，想繳交至少完成其中一項實驗即可
- 請務必自行完成！你可以問 AI，問同學，問學長姊，甚至參考以前別人的作業，但一定要自行 [吸收][執行][整理] 結果！
- 不要貼 AI 的答案寄給老師看，你自己看就好
- 如果系統自動比對發現作業內容 (與他人雷同，原創性低，抄襲比例過高)，作業的分數會 0 分
- 如果你直接 100% 複製別人的作業繳交，你會直接被當掉 (提供作業給別人複製的也一樣)
- 老師看你作業要花時間，真的有心想寫加分作業，請好好自己做

---
### **實驗 1: 多種故障類型的全面分析**

**要求**：
- 載入至少 5 種不同的故障場景數據 (IDV 1~21 中任選)
- 使用 PCA 將所有數據 (正常 + 5 種故障) 降至 2D 並視覺化
- 分析哪些故障類型容易被區分，哪些難以區分
- 使用分群方法 (如 K-Means 或 GMM) 嘗試自動識別不同故障類型
- 計算分群結果與真實故障標籤的一致性 (可使用 Adjusted Rand Index 或 Normalized Mutual Information)
- 建立混淆矩陣，分析哪些故障類型容易被混淆

**分析重點**：
- 不同故障類型在 PCA 空間中的分布特徵
- 無監督分群能否有效識別故障類型
- 製程工程師如何利用這些結果設計故障診斷系統

In [ ]:
# 你的程式碼



### **實驗 2: 動態 PCA 與時間序列分析**

**要求**：
- 實作 **滑動窗口 PCA (Moving Window PCA)**
  - 使用固定大小的時間窗口 (如 50 個時間點)
  - 在每個窗口內計算 T² 和 SPE 統計量
  - 繪製動態監控圖，觀察統計量隨時間的變化
- 分析故障發生的時間點
  - 故障注入後多久能被檢測到 (Detection Delay)
  - 誤報率 (False Alarm Rate) 是多少
- 比較靜態 PCA 與動態 PCA 的檢測性能

**分析重點**：
- 動態 PCA 如何捕捉製程的動態特性
- 時間窗口大小如何影響檢測性能
- 如何權衡檢測靈敏度與誤報率

In [ ]:
# 你的程式碼



### **實驗 3: 進階降維方法比較**

**要求**：
- 實作至少 3 種進階降維方法：
  - **Kernel PCA** (使用 RBF 或 Polynomial kernel)
  - **Autoencoder** (使用 TensorFlow/Keras 建立簡單的自編碼器)
  - **Sparse PCA** 或 **Incremental PCA**
- 比較這些方法與標準 PCA、t-SNE、UMAP 的差異
- 分析每種方法的優缺點：
  - 視覺化效果
  - 計算時間
  - 可解釋性
  - 重建誤差 (對於 PCA 和 Autoencoder)
- 繪製重建誤差圖，比較不同方法的壓縮效果

**分析重點**：
- 非線性降維 (Kernel PCA, Autoencoder) vs 線性降維 (PCA)
- 哪種方法最適合 TEP 數據的特性
- 如何根據應用場景選擇降維方法

In [ ]:
# 你的程式碼



### **實驗 4: 操作變數關聯規則挖掘**

**要求**：
- 將連續型的操作變數 (XMV 1~11) 離散化
  - 可使用等寬分箱 (Equal-Width Binning) 或等頻分箱 (Equal-Frequency Binning)
  - 定義操作區間，如 "Low", "Medium", "High"
- 使用 Apriori 或 FP-Growth 算法挖掘關聯規則
  - 分析正常操作模式下的典型操作變數組合
  - 找出高 Support 和 Confidence 的規則
- 比較不同操作模式 (如不同群集) 的關聯規則差異
- 視覺化關聯規則網絡圖

**分析重點**：
- 哪些操作變數組合經常同時出現
- 這些規則的化工意義是什麼
- 如何利用這些規則指導製程操作

In [ ]:
# 你的程式碼



---
## 💭 思考題

1. **為什麼需要多種非監督式學習方法組合使用？** 單一方法的局限性是什麼？

2. **PCA 降維後，主成分的物理意義是什麼？** 如何向製程工程師解釋第一主成分代表什麼？

3. **在異常檢測中，為什麼要同時使用 T² 和 SPE 兩種統計量？** 它們分別捕捉什麼類型的異常？

4. **如何確定 PCA 保留的主成分數量？** 85% 累積解釋變異量是否總是合理的閾值？

5. **DBSCAN 和 K-Means 在 TEP 數據上的表現有何差異？** 為什麼？

6. **如何設定異常檢測的告警閾值？** 如何權衡誤報率和漏報率？

7. **在製程工業中，為什麼貢獻圖 (Contribution Plot) 很重要？** 它如何幫助製程工程師快速定位故障根因？

8. **如果 PCA 無法有效區分正常和故障狀態，可能的原因是什麼？** 應該嘗試哪些改進方法？

9. **動態製程數據與靜態數據有何不同？** 為什麼需要動態 PCA 或時間序列分析方法？

10. **如何將這些無監督式學習方法整合到實際的製程監控系統中？** 需要考慮哪些工程實務問題？

### 你的思考與回答

1. 

2. 

3. 

4. 

5. 

6. 

7. 

8. 

9. 

10. 

---
## 想對老師說的話